In [ ]:
# if on google collab
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Lol/models')
import c2v
import winner

data_path = '/content/drive/My Drive/Lol/data'
model_path = '/content/drive/My Drive/Lol/models'
c2v_name = 'c2v5'
experiment_name = 'winner_with_both_path2'

Mounted at /content/drive


In [3]:
import numpy as np
import logging
import pandas as pd
import torch
import torch.nn as nn
from torch.distributions import Categorical
from abc import ABC

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

## memory

In [1]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    
    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

## env

In [14]:
class MatchMaker:
  def __init__(self, agent1, agent2, action_dim, judge):
    self.agent1 = agent1
    self.agent2 = agent2
    self.action_dim = action_dim
    self.turns_teams = [1,-1,1,-1,1,-1, 1, -1, -1, 1 ,1 ,-1, -1, 1, -1, 1, -1, 1, 1, -1]
    self.turns_action = ['B'] * 6 + ['P']*6 + ['B']*4 + ['P']*4
    self.judge = judge
    self.reset()


  def reset(self):
    self.state = np.zeros(shape = (20, self.action_dim))
    self.agent1.reset()
    self.agent2.reset()

    # 1: start, -1: end
    self.starter = np.random.choice([-1, 1])
    self.agent1.set_side(self.starter)
    self.agent2.set_side(-self.starter)
    self.agents = {
        1: [self.agent1, self.agent2][int((self.starter+1)/2)],
        -1: [self.agent1, self.agent2][int((-self.starter+1)/2)]
      }
    self.turn = 0
    self.available_champions = [_ for _ in np.arange(action_dim)]

    return self.state

  def step(self):
    """
    /!\ not gym api like
    """
    if self.turn > len(self.turns_teams):
      return 

    # set up turn variables
    current_team = self.turns_teams[self.turn]
    current_action_type = self.turns_action[self.turn]
    current_agent = self.agents[current_team]

    # store state
    current_agent.memorize_state(self.state)

    # get action and check/update action availability
    action, logprob = current_agent.next_action(self.state, current_action_type)
    assert(action in self.available_champions)
    self.available_champions.remove(champion)
    self.agents[-current_team].remove_choice(action)

    # one hot champion ID to update env state
    state_turn = np.zeros(self.action_dim)
    state_turn[action] = 1
    self.state[self.turn, :] = state_turn

    self.turn += 1
    done, reward = self.find_winner_proba()


    # store remaining info
    current_agent.memorize_reste(action, logprob, reward, done)
    # correct done and reawrd info if over
    if done:
      self.agents[1].memory.rewards[-1] = reward
      self.agents[1].memory.is_terminals[-1] = done
      
      self.agents[-1].memory.rewards[-1] = -reward
      self.agents[-1].memory.is_terminals[-1] = done

    return 


  def find_winner_proba(self):
    if self.turn > len(self.turns_teams):
      blue_team = self.state[[7,10,11,18,19]].sum(axis=0)
      res_team = self.state[[8,9,12,17,20]].sum(axis=0)
      return 1, judge.predict([blue_team, red_team])
    else:
      return 0, 0


  def learn(self, max_episodes, update_timestep, update_episode):
    running_reward = 0
    avg_length = 0
    for i_episode in range(1, max_episodes+1):
        state = env.reset()
        for t in range(len(turns_teams)):

            self.step()
            

            
        # update if its time
        if i_episode % update_episode == 0:
            for key, agent in self.agents.items():
              agent.train()
       
        # stop training if avg_reward > solved_reward
        if running_reward > (log_interval*solved_reward):
            print("########## Solved! ##########")
            torch.save(ppo.policy.state_dict(), './PPO_{}.pth'.format(env_name))
            break
            
        # logging
        if i_episode % log_interval == 0:
            avg_length = int(avg_length/log_interval)
            running_reward = int((running_reward/log_interval))
            
            print('Episode {} \t avg length: {} \t reward: {}'.format(i_episode, avg_length, running_reward))
            running_reward = 0
            avg_length = 0

## agent

In [ ]:
class DraftAgent:
  def __init__(self, action_dim):
    self.side = None
    self.action_dim = action_dim
    self.memory = Memory()
    self.reset()

  def reset(self):
    self.possible_choice = [_ for _ in np.arange(action_dim)]
    self.composition = []
    

  def remove_choice(self, champion):
    self.possible_choice.remove(champion)

  @abstractmethod
  def _internal_logic(self, state, action_type):
    pass

  def train(self):
    self.memory.clear_memory()
    pass

  def next_action(self, state, action_type):
    """
    return a champion ID
    """ 
    champion_chosen, logprob = self._internal_logic(state, action_type)
    self.remove_choice(champion_chosen)
    if action_type = 'P':
      self.composition.append(champion_chosen)
    return champion_chosen, logprob

  def set_side(self, side):
    self.side=side

  def memorize_state(self,  state):
    self.memory.states.append(state)

  def memorize_reste(self, action, logprob, reward, done):
    self.memory.actions.append(action)
    self.memory.logprobs.append(logprob)
    self.memory.rewards.append(reward)
    self.memory.is_terminals.append(done)

def RandomAgent(DraftAgent):
  def _internal_logic(self, state):
    return np.random.choice(self.possible_choice), 0

def PPOAgent(DraftAgent):
  def __init__(self, action_dim, ppo):
    super().__init__(self, action_dim)
    self.ppo = ppo

  def _internal_logic(self, state, aciton_type):
                # Running policy_old:
            action = ppo.policy_old.act(state, memory)
            state, reward, done, _ = env.step(action)



In [16]:
class ActorCritic(nn.Module):
    """
    from https://github.com/nikhilbarhate99/PPO-PyTorch/blob/master/test_continuous.py
    """
    def __init__(self, state_dim, action_dim, n_latent_var):
        super(ActorCritic, self).__init__()

        # actor
        self.action_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, action_dim),
                nn.Softmax(dim=-1)
                )
        
        # critic
        self.value_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, 1)
                )
        
    def forward(self):
        raise NotImplementedError
        
    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device) 
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))
        
        return action.item()
    
    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        
        state_value = self.value_layer(state)
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy
        
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()
    
    def update(self, memory):   
        # Monte Carlo estimate of state rewards:
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        
        # Normalizing the rewards:
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        
        # convert list to tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()
        
        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):
            # Evaluating old actions and values :
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            
            # Finding the ratio (pi_theta / pi_theta__old):
            ratios = torch.exp(logprobs - old_logprobs.detach())
                
            # Finding Surrogate Loss:
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
        
        # Copy new weights into old policy:
        self.policy_old.load_state_dict(self.policy.state_dict())